# Momentum 策略

本專案旨在系統性地重建並驗證美股市場中動能投資組合 (Momentum Portfolios) 的表現與穩健性。透過使用 CRSP 股票資料、Fama-French 因子檔以及外部提供的 decile 報酬率檔 (DM 與 KRF)，本專案執行以下主要步驟：

1. 資料前處理與清理

    對 CRSP 股票資料進行篩選（只留普通股與主要交易所股票），合併正常報酬與退市報酬，確保資料正確性。
    
    計算 lag 市值，避免 look-ahead bias。

2. 動能因子計算

    以 12-2 月累積報酬作為 Ranking Return，根據全市場 (DM) 與 NYSE-only (KRF) 分組基準，將股票分成 10 個 decile。

3. 投資組合報酬計算

    根據 lagged 市值加權，計算每個 decile 的每月報酬率。
    
    計算動能投資組合 (WML: Winner-Minus-Loser) 報酬，捕捉動能異象的 alpha。

4. 績效與穩健性驗證

    與 Daniel & Moskowitz、Kenneth R. French 官方 decile 報酬率檔案進行相關性檢驗。
    
    輸出 decile & WML 投資組合的年化報酬、波動度、Sharpe Ratio、偏態與與官方資料的相關性。

5. 視覺化與結論

    繪製 1929-2024 & 2010-2024 年間的累積 log 報酬率圖，直觀比較動能策略在不同時期的表現。
    
    提供研究洞察與未來可能的市場 regime shift 影響。

In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [59]:
# 對於原始的 CRSP 股票資料，進行初步的資料清理與格式標準化。
def clean_CRSP_Stocks(df: pd.DataFrame):
    # Casting date to datetime
    df_clean = df.copy()
    df_clean.date = pd.to_datetime(df_clean.date, format='%Y-%m-%d')
    df_clean.EXCHCD = df_clean.loc[:, "EXCHCD"].astype("int64")

    # Filterling only common share class (10, 11) and  major exchange(1, 2, 3)
    df_clean = df_clean[df_clean.EXCHCD.isin([1, 2, 3]) & df_clean.SHRCD.isin([10, 11])]
    
    # replace all non-numeric (including nan) value to zero
    df_clean.loc[:, "valid_ret"] = ~(df_clean.RET.isna() & df_clean.DLRET.isna())
    
    df_clean.DLRET = df_clean.DLRET.fillna(0)
    df_clean.DLRET = df_clean.DLRET.replace(["S", "T", "A", "P"], 0).astype(float)
    df_clean.RET = df_clean.RET.fillna(0)
    df_clean.RET = df_clean.RET.replace(["A", "B", "C", "D", "E", "."], 0).astype(float)
    
    # Merging Delisting and Holding Returns
    df_clean.RET = (1+df_clean.RET) * (1+df_clean.DLRET) - 1

    # Sorting the dataframe by date and premno
    df_clean.sort_values(["date", "PERMNO"], ignore_index=True, inplace=True)
    df_clean.drop(["SHRCD", "DLRET"], axis=1, inplace=True)
    
    # Change datatype for EXCHCD
    df_clean.loc[:, "EXCHCD"] = df_clean.loc[:, "EXCHCD"].astype(np.int8)
    
    # Remove Null Quote
    return df_clean.rename({"RET":"Ret"}, axis=1)

In [61]:
#計算市值與上一期市值以計算市值權重(避免look-ahead bias)
def get_mkt_cap(df: pd.DataFrame):
    df_mkt_cap = df.copy()
    # Calculate Market Cap for each stock 
    df_mkt_cap.loc[:, "mkt_cap"]= np.abs(df_mkt_cap.PRC) * df_mkt_cap.SHROUT / 1e3
    
    # Calculate Lagged Market Cap for weight calculations
    def get_lagged_mkt_cap(df):
        df = df.assign(
            month_diff = lambda x: (x.date.dt.year - x.date.shift(1).dt.year) * 12 + x.date.dt.month - x.date.shift(1).dt.month,
            lag_Mkt_Cap = lambda x: (x.mkt_cap.shift(1) * (x.month_diff==1))
        ).drop("month_diff", axis=1)
        return df
    
    df_mkt_cap = df_mkt_cap.groupby(["PERMNO"]).apply(get_lagged_mkt_cap, include_groups=False).reset_index().drop("level_1", axis=1)
    return df_mkt_cap

In [63]:
#計算動能排序來分組
def get_ranking_returns(df: pd.DataFrame):
    df.sort_values(["PERMNO", "date"], inplace=True)
    unique_date = df.sort_values("date").date.unique()
    date = df.date
    # Check constraints
    df = df.assign(
        valid_prc = lambda x: x.PRC.notna(),
        # Floor return to avoid errors when converting to log returns
        floor_Ret = lambda x: np.maximum(x.Ret, -0.99999),
        log_Ret = lambda x: np.log1p(x.floor_Ret),
        valid_shr = lambda x: x.SHROUT.notna(),
        valid_mktcap = lambda x: x.lag_Mkt_Cap.notna() & x.lag_Mkt_Cap > 0,
        valid_formation = lambda x: x.valid_prc & x.valid_shr & x.valid_mktcap
    )
    res = []
    
    for start, ret_start, ret_end, end in zip(unique_date, unique_date[1:], unique_date[11:], unique_date[13:]):
        df_filtered = df.loc[(date>=start)&(date<=end)]

        # Filter out all ineligible stocks based on the requirements
        m_1 = (df_filtered.date==start)&(df_filtered.valid_prc)
        eligible_permno_1 = set(df_filtered.loc[m_1, "PERMNO"])
        
        m_2 = (df_filtered.date==ret_end)&(df_filtered.valid_ret)
        eligible_permno_2 = set(df_filtered.loc[m_2, "PERMNO"])
        
        m_3 = (df_filtered.date==end)&(df_filtered.valid_formation)
        eligible_permno_3 = set(df_filtered.loc[m_3, "PERMNO"])
        
        eligible_permno_4 = df_filtered.loc[(df_filtered.date>=ret_start)&(df_filtered.date<=ret_end)].groupby(["PERMNO"])['valid_ret'].sum().ge(8).reset_index()
        eligible_permno_4 = set(eligible_permno_4.loc[eligible_permno_4.valid_ret, "PERMNO"])
        
        eligible_permno = eligible_permno_1 & eligible_permno_2 & eligible_permno_3 & eligible_permno_4
        ret_t = df_filtered[(df_filtered.date>=ret_start) & (df_filtered.PERMNO.isin(eligible_permno))]

        cum_rets = ret_t[ret_t.date<=ret_end].groupby(["PERMNO"])["log_Ret"].sum().rename("Ranking_Ret")

        ret_t  = pd.merge(ret_t.loc[ret_t.date==end, :], cum_rets, how="inner", left_on=["PERMNO"], right_index=True)
        res.append(ret_t)
        
    res = pd.concat(res) 
    res = res.assign(
            Year = res.date.dt.year,
            Month = res.date.dt.month
        ).drop("date", axis=1)
    
    
    return res.reset_index().loc[:, ["Year", "Month", "PERMNO", "EXCHCD", "lag_Mkt_Cap", "Ret", "Ranking_Ret"]]

In [65]:
def prepare_momentum_data(CRSP_Stocks: pd.DataFrame):
    CRSP_Stocks_clean = clean_CRSP_Stocks(CRSP_Stocks)
    CRSP_Stocks_mkt_cap = get_mkt_cap(CRSP_Stocks_clean)
    CRSP_Stocks_Momentum = get_ranking_returns(CRSP_Stocks_mkt_cap)
    
    return CRSP_Stocks_Momentum

In [67]:
# 分組(paper vs French)
def assign_momentum_deciles(CRSP_Stocks_Momentum: pd.DataFrame):
    def get_decile(x):
        # Daniel & Mokskowitz
        # Get Breakpoints based on all Stocks
        x.loc[:, "DM_decile"] = pd.qcut(x.Ranking_Ret, 10, labels=np.arange(1, 11, 1), precision=5).astype(int)
        # Kenneth R. French (Below code are from Momentum_weekly_FULL_CODE.ipynb provided by the professor)
        # Get Breakpoints based on only NYSE Stocks (Exchange Code = 1)
        nyse_breakpoint = pd.qcut(x.loc[x.EXCHCD==1, "Ranking_Ret"], 10, retbins=True, labels=False)[1]
        nyse_breakpoint[0], nyse_breakpoint[10] = -np.inf, np.inf
        x.loc[:, "KRF_decile"] = pd.cut(x.Ranking_Ret, bins=nyse_breakpoint, labels=np.arange(1, 11, 1), precision=5).astype(int)
        return x
    
    CRSP_Stocks_Momentum_decile = CRSP_Stocks_Momentum.groupby(["Year", "Month"]).apply(get_decile, include_groups=False).reset_index()
    return CRSP_Stocks_Momentum_decile.loc[:, ["Year", "Month", "PERMNO", "lag_Mkt_Cap", "Ret", "DM_decile", "KRF_decile", "EXCHCD"]]


In [69]:
#清理Fama-French資料
def clean_FF_mkt(df: pd.DataFrame):
    df_clean = df.copy()
    df_clean = df_clean.assign(
        Date = lambda x: pd.to_datetime(x.iloc[:, 0], format='%Y%m'),
        Year = lambda x: x.Date.dt.year,
        Month = lambda x: x.Date.dt.month,
    )
    df_clean.loc[:, ["Mkt-RF", "SMB", "HML", "RF"]] = df_clean.loc[:, ["Mkt-RF", "SMB", "HML", "RF"]]/100
    # Rearranging and dropping the columns
    df_clean = df_clean.loc[:, ["Year", "Month", "Mkt-RF", "SMB", "HML", "RF"]]
    df_clean.columns = ["Year", "Month", "Market_minus_Rf", "SMB", "HML", "Rf"]
    return df_clean

In [71]:
#計算每個月時每個decile投資組合的市值加權報酬率
def calculate_portfolio_returns(CRSP_Stocks_Momentum_decile: pd.DataFrame, FF_mkt: pd.DataFrame):
    def get_vw_rets(x):
        x.loc[:, "total_mv"] = x.lag_Mkt_Cap.sum()
        x.loc[:, "Port_Ret"] = np.sum(x.Ret * x.lag_Mkt_Cap) / x.total_mv
        return x[["Port_Ret"]].iloc[-1]
    
    # Get Value-weighted return for Daniel & Mokskowitz Portfolio
    DM_ret = CRSP_Stocks_Momentum_decile.groupby(["Year", "Month", "DM_decile"]).apply(get_vw_rets, include_groups=False).reset_index()
    DM_ret = DM_ret.loc[:, ["Year", "Month", "DM_decile", "Port_Ret"]].rename({"DM_decile":"decile", "Port_Ret":"DM_Ret"}, axis=1)
    
    # Get Value-weighted return for Kenneth R. French
    KRF_ret = CRSP_Stocks_Momentum_decile.groupby(["Year", "Month", "KRF_decile"]).apply(get_vw_rets, include_groups=False).reset_index()
    KRF_ret = KRF_ret.loc[:, ["Year", "Month", "KRF_decile", "Port_Ret"]].rename({"KRF_decile":"decile", "Port_Ret":"KRF_Ret"}, axis=1)
    
    # Extract Rf rate for each month
    Rf = FF_mkt.loc[:, ["Year", "Month", "Rf"]]
    Rf.loc[:, "Rf"] = Rf.loc[:, "Rf"]
    
    # Merge value returns for both portfolio and risk free rate
    CRSP_Stocks_Momentum_returns = pd.merge(DM_ret, KRF_ret, how="inner", on=["Year", "Month", "decile"])
    CRSP_Stocks_Momentum_returns = pd.merge(CRSP_Stocks_Momentum_returns, Rf, how="left", on=["Year", "Month"])
    
    return CRSP_Stocks_Momentum_returns

In [73]:
#將paper提供的報酬檔案做清理與標準化, to compare
def clean_DM_ret(df: pd.DataFrame):
    df_clean = df.copy()
    df_clean.columns = ["date", "decile", "DM_Ret", "_", "__"]
    df_clean = df_clean.assign(
        Date = lambda x: pd.to_datetime(x.iloc[:, 0], format='%Y%m%d'),
        Year = lambda x: x.Date.dt.year,
        Month = lambda x: x.Date.dt.month,
    )
    # Rearranging and dropping the columns
    return df_clean.loc[:, ["Year", "Month", "decile", "DM_Ret"]]

In [75]:
#計算每個月winner-loser的報酬率
def get_wml(df: pd.DataFrame, col: str):
        # Pivot the dataframe
        pivot = df.pivot(index=["Year","Month"], columns="decile", values=col)
        # Derive WML returns
        pivot.loc[:, "WML"] = pivot[10] - pivot[1]
        # Restore the original format
        pivot = pivot.reset_index().melt(["Year","Month"], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, "WML"], "decile", col)
        return pivot

In [77]:
#計算我自己複製的DM分組中每個Decile的年度化報酬、波動度、sharpe ratio、偏態以及和原始DM檔的相關性
#驗證有沒有高度一致及看風險調整後表現
def generate_dm_summary_stats(CRSP_Stocks_Momentum_returns: pd.DataFrame, DM_returns: pd.DataFrame):
    
    def get_summary_stats(x: pd.Series):
        # Annualized Mean
        ann_mean = x.DM_Ret_replica.mean()*12
        # Standard Deviation
        ann_std = x.DM_Ret_replica.std()*np.sqrt(12)
        # Sharpe Ratio
        sr = ann_mean/ann_std
        # Skewness
        skewness = np.log(x.DM_Ret_replica+x.Rf+1).skew()
        # correlation
        corr = x.loc[:, ["DM_Ret_replica", "DM_Ret_true"]].corr().loc["DM_Ret_replica", "DM_Ret_true"]
        return [round(ann_mean*100, 2), round(ann_std*100, 2), round(sr, 2), round(skewness, 2), round(corr, 4)]
    
    df = CRSP_Stocks_Momentum_returns.copy()
    rf = CRSP_Stocks_Momentum_returns.loc[:, ["Year", "Month", "Rf"]]
    
    rf = rf.groupby(["Year", "Month"])["Rf"].first().reset_index()
    df = get_wml(df, "DM_Ret")
    
    df = pd.merge(df, rf, how="left", on=["Year", "Month"])
    DM_returns = get_wml(DM_returns, "DM_Ret")
    
    df = pd.merge(df, DM_returns, how="left", left_on=["Year", "Month", "decile"], right_on=["Year", "Month", "decile"], suffixes=("_replica", "_true"))
    
    df.loc[df.decile!="WML", "DM_Ret_replica"] = df.loc[df.decile!="WML", "DM_Ret_replica"] - df.loc[df.decile!="WML", "Rf"]
    df.loc[df.decile!="WML", "DM_Ret_true"] = df.loc[df.decile!="WML", "DM_Ret_true"] - df.loc[df.decile!="WML", "Rf"]
    
    res_index = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, "WML"]
    res = [get_summary_stats(df.loc[df.decile==decile, :]) for decile in res_index]
    res_col = ["Excess Return", "Volatility", "Sharpe Ratio", "Skewness", "corr w/ original"]
    res = pd.DataFrame(res, index=res_index, columns=res_col)
    return res.T

In [79]:
#整理KRF decile報酬
def clean_KRF_ret(df: pd.DataFrame):
    df_clean = df.copy()
    df_clean.columns = np.arange(1, 11, 1)
    df_clean = df_clean.assign(
        Date = lambda x: pd.to_datetime(x.index, format='%Y%m'),
        Year = lambda x: x.Date.dt.year,
        Month = lambda x: x.Date.dt.month,
    ).reset_index(drop=True).drop("Date", axis=1)
   
    # Melt the columns to row
    df_clean = df_clean.melt(["Year","Month"], np.arange(1, 11, 1), "decile", "KRF_Ret")
    df_clean.loc[:, "KRF_Ret"] = df_clean.loc[:, "KRF_Ret"]/100
    return df_clean

In [81]:
#計算我複製出來的KRF decile投資組合的指標
def generate_krf_summary_stats(CRSP_Stocks_Momentum_returns: pd.DataFrame, KRF_returns: pd.DataFrame):
    def get_summary_stats(x: pd.Series):
        # Annualized Mean
        ann_mean = x.KRF_Ret_replica.mean()*12
        # Standard Deviation
        ann_std = x.KRF_Ret_replica.std()*np.sqrt(12)
        # Sharpe Ratio
        sr = ann_mean/ann_std
        # Skewness
        skewness = np.log(x.KRF_Ret_replica+x.Rf+1).skew()
        # correlation
        corr = x.loc[:, ["KRF_Ret_replica", "KRF_Ret_true"]].corr().loc["KRF_Ret_replica", "KRF_Ret_true"]
        return [round(ann_mean,4)*100, round(ann_std,4)*100, round(sr, 2), round(skewness, 2), round(corr, 4)]
    
    df = CRSP_Stocks_Momentum_returns.copy()
    rf = CRSP_Stocks_Momentum_returns.loc[:, ["Year", "Month", "Rf"]]
    
    rf = rf.groupby(["Year", "Month"])["Rf"].first().reset_index()
    df = get_wml(df, "KRF_Ret")
    
    df = pd.merge(df, rf, how="left", on=["Year", "Month"])
    KRF_returns = get_wml(KRF_returns, "KRF_Ret")
    
    df = pd.merge(df, KRF_returns, how="left", left_on=["Year", "Month", "decile"], right_on=["Year", "Month", "decile"], suffixes=("_replica", "_true"))
    
    df.loc[df.decile!="WML", "KRF_Ret_replica"] = df.loc[df.decile!="WML", "KRF_Ret_replica"] - df.loc[df.decile!="WML", "Rf"]
    df.loc[df.decile!="WML", "KRF_Ret_true"] = df.loc[df.decile!="WML", "KRF_Ret_true"] - df.loc[df.decile!="WML", "Rf"]
    
    res_index = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, "WML"]
    res = [get_summary_stats(df.loc[df.decile==decile, :]) for decile in res_index]
    res_col = ["Excess Return", "Volatility", "Sharpe Ratio", "Skewness", "corr w/ original"]
    res = pd.DataFrame(res, index=res_index, columns=res_col)
    return res.T

In [ ]:
if __name__ == "__main__":
    # Question 1
    CRSP_Stocks = pd.read_pickle('./data/CRSP.pkl')
    CRSP_Stocks = CRSP_Stocks.rename({"permno":"PERMNO", "exchcd":"EXCHCD", "ret":"RET", "shrcd":"SHRCD", "shrout":"SHROUT", "prc":"PRC", "dlret":"DLRET"}, axis=1)
    CRSP_Stocks = CRSP_Stocks.loc[:, ["date", "PERMNO", "EXCHCD", "RET", "SHRCD", "SHROUT", "PRC", "DLRET"]]
    CRSP_Stocks_Momentum = prepare_momentum_data(CRSP_Stocks)
    print("Sample Result for Question 1")
    print(CRSP_Stocks_Momentum)
    print("")

    # Question 2
    CRSP_Stocks_Momentum_decile = assign_momentum_deciles(CRSP_Stocks_Momentum)
    print("Sample Result for Question 2")
    print(CRSP_Stocks_Momentum_decile)
    print("")

    # Question 3
    FF_mkt = pd.read_csv("F-F_Research_Data_Factors.CSV", skiprows=3, nrows=1182)
    FF_mkt = clean_FF_mkt(FF_mkt)
    print("Input: Fama-French Market DataFrame (FF_mkt)")
    print(FF_mkt)
    print("")
    
    CRSP_Stocks_Momentum_returns = calculate_portfolio_returns(CRSP_Stocks_Momentum_decile, FF_mkt)
    print("Sample Result for Question 3")
    print(CRSP_Stocks_Momentum_returns)
    print("")

    # Question 4
    DM_returns = pd.read_csv('m_m_pt_tot.txt', sep="\\s+", header=None)
    DM_returns = clean_DM_ret(DM_returns)
    print("Input: Momentum portfolio returns from Daniel\'s website (DM_returns)")
    print(DM_returns)
    print("")

    print("Sample Result for Question 4")
    DM_res = generate_dm_summary_stats(CRSP_Stocks_Momentum_returns, DM_returns)
    print(DM_res)
    print("")

    # Question 5
    KRF_returns = pd.read_csv("10_Portfolios_Prior_12_2.csv", skiprows=10, nrows=1176, index_col=0)
    KRF_returns = clean_KRF_ret(KRF_returns)
    print("Input: Momentum portfolio returns from French\'s website (KRF_returns)")
    print(KRF_returns)
    print("")

    print("Sample Result for Question 5")
    KRF_res = generate_krf_summary_stats(CRSP_Stocks_Momentum_returns, KRF_returns)
    print(KRF_res)
    print("")

    # Question 6
    wml_port_rets = get_wml(CRSP_Stocks_Momentum_returns, "DM_Ret").merge(get_wml(CRSP_Stocks_Momentum_returns, "KRF_Ret"), on=["Year", "Month", "decile"])
    wml_port_rets = wml_port_rets[wml_port_rets.decile=="WML"].drop(["decile"], axis=1)
    wml_port_rets.loc[:, "str_date"] = wml_port_rets["Year"].astype(str) + wml_port_rets["Month"].astype(str).str.zfill(2)
    wml_port_rets.loc[:, "date"] = pd.to_datetime(wml_port_rets.loc[:, "str_date"], format='%Y%m')
    wml_port_rets = wml_port_rets.set_index("date")
    wml_port_rets.drop(["Year", "Month", "str_date"], axis=1, inplace=True)
    wml_port_rets = np.log(wml_port_rets+1).cumsum()
    wml_port_rets.columns = ["DM", "KRF"]
    wml_port_rets.plot()
    plt.title("Cumulative Log Returns of MOM portfolios 1929-2024")
    plt.show()

    wml_port_rets_2010 = wml_port_rets.loc["2010-01-01":]
    wml_port_rets_2010 = wml_port_rets_2010 - wml_port_rets_2010.loc["2010-01-01", :]
    wml_port_rets_2010.plot()
    plt.title("Cumulative Log Returns of MOM portfolios 2010-2024")
    plt.show()

